<a href="https://colab.research.google.com/github/reallylongaddress/abnb_preprocessor/blob/master/notebooks/abnb_lookup_municipality.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !pip install geopy

import numpy as np
import pandas as pd
from google.colab import drive

from geopy.geocoders import Nominatim

In [2]:
drive = drive.mount('/content/drive/')
data_path = '/content/drive/My Drive/Colab_Data/AirBnb/Italy/Sicily/'

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [3]:
ld_file_path = data_path + 'detailed_listings.csv'
ld_df_clean = pd.read_csv(ld_file_path, nrows=5)

ld_df_clean.columns

Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'name', 'description',
       'neighborhood_overview', 'picture_url', 'host_id', 'host_url',
       'host_name', 'host_since', 'host_location', 'host_about',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_thumbnail_url', 'host_picture_url',
       'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'neighbourhood',
       'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'latitude',
       'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms',
       'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price',
       'minimum_nights', 'maximum_nights', 'minimum_minimum_nights',
       'maximum_minimum_nights', 'minimum_maximum_nights',
       'maximum_maximum_nights', 'minimum_nights_avg_ntm',
       'maximum_nights_avg_ntm', 'calendar_upd

In [35]:
ld_df = ld_df_clean.copy()[['id', 'listing_url', 'latitude', 'longitude']]
ld_df.head()

,id,listing_url,latitude,longitude
0,207654,https://www.airbnb.com/rooms/207654,37.61484,15.01867
1,125569,https://www.airbnb.com/rooms/125569,37.61566,15.01782
2,702537,https://www.airbnb.com/rooms/702537,37.57494,12.74790
3,230912,https://www.airbnb.com/rooms/230912,38.17447,12.75140
4,137342,https://www.airbnb.com/rooms/137342,36.92940,14.62523


In [36]:
geolocator = Nominatim(user_agent="specify_your_app_name_here")

In [37]:
# #what are all the different potential address components?
# field_list = []

# for ind in ld_df.index:

#   location = geolocator.reverse(f"{ld_df['latitude'][ind]}, {ld_df['longitude'][ind]}")

#   address_dict = location.raw.get('address', {})
#   field_list.extend(list(address_dict.keys()))

# pd.DataFrame(field_list).value_counts()

In [38]:
'''
FIND

state                300
country_code         300
county               300
postcode             278
road                 248
town                 139
village              138
city                  80
suburb                58
hamlet                53
neighbourhood         51
quarter               15
municipality           3


state, country_code, county, postcode, road, town, village, city, suburb, hamlet,neighbourhood, quarter, municipality
'state', 'country_code', 'county', 'postcode', 'road', 'town', 'village', 'city', 'suburb', 'hamlet', 'neighbourhood', 'quarter', 'municipality'

''';

In [39]:
#DBD TODO:  store this data frame so we don't have to keep looking this info up (it won't change)
abnb_geo_df = pd.DataFrame(columns=['geo_state', 'geo_country_code', 'geo_county', 'geo_postcode', 'geo_road', 'geo_town', 'geo_village', 'geo_city', 'geo_suburb', 'geo_hamlet', 'geo_neighbourhood', 'geo_quarter', 'geo_municipality'])

#what are all the different potential address components?
address_field_list = []

for ind in ld_df.index:

  location = geolocator.reverse(f"{ld_df['latitude'][ind]}, {ld_df['longitude'][ind]}")

  address_dict = location.raw.get('address', {})

  abnb_geo_df.loc[len(abnb_geo_df.index)] = [address_dict.get('state', ''),
                                             address_dict.get('country_code', ''),
                                             address_dict.get('county', ''),
                                             address_dict.get('postcode', ''),
                                             address_dict.get('road', ''),
                                             address_dict.get('town', ''),
                                             address_dict.get('village', ''),
                                             address_dict.get('city', ''),
                                             address_dict.get('suburb', ''),
                                             address_dict.get('hamlet', ''),
                                             address_dict.get('neighbourhood', ''),
                                             address_dict.get('quarter', ''),
                                             address_dict.get('municipality', '')]


  address_dict = location.raw.get('address', {})
  address_field_list.extend(list(address_dict.keys()))
  
if True:
  display(pd.DataFrame(address_field_list).value_counts())



ISO3166-2-lvl4       5
ISO3166-2-lvl6       5
country              5
country_code         5
county               5
postcode             5
road                 5
state                5
village              3
city                 1
hamlet               1
isolated_dwelling    1
town                 1
dtype: int64

In [40]:
abnb_geo_df

,geo_state,geo_country_code,geo_county,geo_postcode,geo_road,geo_town,geo_village,geo_city,geo_suburb,geo_hamlet,geo_neighbourhood,geo_quarter,geo_municipality
0,Sicilia,it,Catania,95030,Viale Aldo Moro,,Nicolosi,,,,,,
1,Sicilia,it,Catania,95030,Viale Aldo Moro,,Nicolosi,,,,,,
2,Sicilia,it,Trapani,91021,Viale Nicolò Gentile,Campobello di Mazara,,,,Tre Fontane,,,
3,Sicilia,it,Trapani,91010,Via del Secco,,San Vito Lo Capo,,,,,,
4,Sicilia,it,Ragusa,97013,Strada provinciale Castiglione-Tresauro,,,Ragusa,,,,,


In [43]:
ld_df2 = pd.concat([ld_df, abnb_geo_df.reindex(ld_df.index)], axis=1)
ld_df2

,id,listing_url,latitude,longitude,geo_state,geo_country_code,geo_county,geo_postcode,geo_road,geo_town,geo_village,geo_city,geo_suburb,geo_hamlet,geo_neighbourhood,geo_quarter,geo_municipality
0,207654,https://www.airbnb.com/rooms/207654,37.61484,15.01867,Sicilia,it,Catania,95030,Viale Aldo Moro,,Nicolosi,,,,,,
1,125569,https://www.airbnb.com/rooms/125569,37.61566,15.01782,Sicilia,it,Catania,95030,Viale Aldo Moro,,Nicolosi,,,,,,
2,702537,https://www.airbnb.com/rooms/702537,37.57494,12.74790,Sicilia,it,Trapani,91021,Viale Nicolò Gentile,Campobello di Mazara,,,,Tre Fontane,,,
3,230912,https://www.airbnb.com/rooms/230912,38.17447,12.75140,Sicilia,it,Trapani,91010,Via del Secco,,San Vito Lo Capo,,,,,,
4,137342,https://www.airbnb.com/rooms/137342,36.92940,14.62523,Sicilia,it,Ragusa,97013,Strada provinciale Castiglione-Tresauro,,,Ragusa,,,,,


In [46]:
#save the DF so we don't have to keep doing geo lookups
file_path = data_path + f'abnb_geo_df_{len(ld_df2)}.csv'
ld_df2.to_csv(file_path)

In [10]:
# field_list

## Merge DataFrames (Abnb + Reverse Geo)

In [10]:
ld_df